In [9]:
from package.sphincs import Sphincs
from tqdm import tqdm

### Instantiate a $SPHINCS^+$ Object and Setting parameters ###

We will be using :
   - Security Parameter: $n=2$
   - Winternitz Parameter: $w=16$
   - Hypertree Height: $h=4$
   - Hypertree Layers: $d=2$
   - $FORS$ Trees Number: $k=4$
   - $FORS$ Trees Height: $a=2$

In [10]:
sphincs = Sphincs()

sphincs.set_n(2)
sphincs.set_h(4)
sphincs.set_d(2)
sphincs.set_k(4)
sphincs.set_a(2)

### Generating a Key Pair ###

In [11]:
sk, pk = sphincs.generate_key_pair()
print("Secret Key: ", sk)
print()
print("Public Key: ", pk)

Secret Key:  b'\x87\xd5\xb5LR\xca\xad\xbc]\x1cZ\x07\x0b\x93U>\xe2MJ`mZ\x89X\x11\x90\xfe\xc8\x19e0\x8a5H\xefF\x08\xaeaU_V\x08\xa6@\x93\xc6\x8e\xb6\xe7N6\x1d\x04\xa0\x93b\xccxJ\xe1.\xe2\\\xbc\x95V\xd6K\xe0\xdc\xf83i\x96\x15\xb1\xde]\xc65#C\xc9\x0c5-\x13#\xc1:j46\xb8\xbe\x9ch\xdd)\xeb4\xa3\xac\xe0U\x91\x1eyL&"\x7f\xc4k(\xea\xaaG\x05=+\xb3\xdc\xe2gCg'

Public Key:  b'\xbc\x95V\xd6K\xe0\xdc\xf83i\x96\x15\xb1\xde]\xc65#C\xc9\x0c5-\x13#\xc1:j46\xb8\xbe\x9ch\xdd)\xeb4\xa3\xac\xe0U\x91\x1eyL&"\x7f\xc4k(\xea\xaaG\x05=+\xb3\xdc\xe2gCg'


### Signing M ###

In [12]:
m = b'Ripples of paradox spread out across the sea of causality.'

signature = sphincs.sign(m, sk)

print("Signature Size: ", len(signature))

Signature Size:  7684


In [13]:
print("Is signature Correct ? ", sphincs.verify(signature, m, pk))

Is signature Correct ?  False


### Trying to find secret key with a Brute Force Attack on Secret Key###

In [14]:
sk_crack = bytes()

for i in tqdm(range(0, 2 ** (sphincs._n * 8))):
    sk_crack = i.to_bytes(sphincs._n, 'big')  # Secret Key
    
    sk_crack += bytes(sphincs._n)   # Random Secret PRF, important to prevent forged messages from actual messages
                                    # But Because we are brute forcing Secret Key, messages are forged before
                                    # We don't need to create a really random one (0 is fine)
    sk_crack += pk  # Public Key
    
    sig_crack = sphincs.sign(m, sk_crack)  # Creating a signature
    
    if sphincs.verify(sig_crack, m, pk):  # Check if signature could be trust with the Public Key
        print("Secret Key Found: ", sk_crack, "\nWith main Private Seed: ", sk_crack[:sphincs._n])
        print("Cycles: ", i)
        break

print("\nDid we found the actual Private Seed? ", sk[:sphincs._n] == sk_crack[:sphincs._n])

if sk[:sphincs._n] != sk_crack[:sphincs._n]:
    print("We found a collision with the main seed!")

  0%|          | 16/65536 [00:08<9:25:35,  1.93it/s]


KeyboardInterrupt: 

### Trying to forge message with found Key ###

In [ ]:
m2 = b'The pen is mightier than the sword ... if the sword is very short, and the pen is very sharp.'

signature2 = sphincs.sign(m2, sk_crack)

print(len(signature2))

58


In [ ]:
print("Is signature Correct ? ", sphincs.verify(signature2, m, pk))

Is signature Correct ?  False


Our signature is wrong because we tried finding the secret key using only $m$ signature, with $m_2$, this kind of collision doesn't work !
We need to find the real secret key in order to forge our own message, and so test every possibilities.